In [ ]:
######Brian Snyder 10/8/19  gCOD Sustained Wind Speed
# Updated 10/9/18 Moved to scheduled daily update on IBM Watson Studio
# Gets multiple gCOD Sustained Wind Speed layers from our featurizer catalog and creates separate layers
# Each layer is coded to this array of esri Feature Layer IDs from ARCGIS Online: 
# esriIds = ["d67442e1ceb74c3b8a43c628a7417461","0236b61718aa40479feec2c0107358cd","54d11de9ce9947879c5177e6d2006b76", "08e823a0223c498c80b7697c0687210a","dde7db1f53be46e0a5ed8a53b1fd3475", "cb42cb3ff7a2489bb3dcc0f123b53a7c"]

import requests,json,datetime # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(33) #  gCOD Sustained Wind Speed 
prod_name='Windspeedheightaboveground' # Per TWC naming conventions
APIKEY='THE_WEATHER_COMPANY_WEATHER_DATA_PACKAGE_KEY_HERE'  #Enter your API key 


########################PRODUCT INVENTORY CALL################################
#We get the most recent runtime.
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print("Call #1: " + url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        for x in range(0,1):
            for y in range(0,1):
                timestamp = data["layers"][prod_num][prod_name]["dimensions"][x]["t"][y]
            return timestamp

##########################   API CALL   ###################################
def APIcall(timestamp,ts): # function for pulling data 
    count=0
    num_layers = 6  
    inc=5           
    low_limit = 5   ##Thresholds are in F.  Starting with negative
    low_limitS = str(low_limit)
    high_limit=low_limit+5
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'
    # First, run this file to get the initial geosjon files.
    # Second, upload them one at a time as a feature layer in ARCGIS Online.
    # Third, grab each of the layer ids and update them below.
    # Fourth, run this script with the new codes to update at a frequenciy of your choice.  You may consider IBM Watson Studio to schedule the updates.
    esriIds = ["d67442e1ceb74c3b8a43c628a7417461","0236b61718aa40479feec2c0107358cd","54d11de9ce9947879c5177e6d2006b76", "08e823a0223c498c80b7697c0687210a","909dc0568d8943ff856ad98f804aead9", "c9bd84434c704b269dcc0c09abe6096c"]
    for x in range(0,num_layers):
        fc = {
            "type": "FeatureCollection",
            "features": []
        }
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print ("Call #2: " + url) 
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']            
            sustainedWindSpeed = str(y['properties']['threshold'])
            f['properties']['Sustained Wind Speed'] = sustainedWindSpeed  
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
          
        
        #Update each piece of content in ESRI        
        with open("gCOD_Sustained_Wind_Speed_" + str(low_limitS) + ".geojson", "w") as outfile:
            json.dump(fc, outfile)
        if(len( fc['features']) != 0):
            portalURL = "http://ibmmain.maps.arcgis.com"
            userID = "ESRI_USERNAME"
            password = "ESRI_PASSWORD"
            gis = GIS(portalURL, userID, password)
            existing_ID = str(esriIds[x])  # Map ID
            wx_item = gis.content.get(existing_ID)
            collection = features.FeatureLayerCollection.fromitem(wx_item)
            collection.manager.overwrite('gCOD_Sustained_Wind_Speed_' + str(low_limitS) + '.geojson')  
            print ("Updating Layer: gCOD_Sustained_Wind_Speed_" + str(low_limitS) + '.geojson')
        else:
            print ("NOT Updating Layer: gCOD_Sustained_Wind_Speed_" + str(low_limitS) + '.geojson as since no results for this threshold' )

        #reset limits
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit)

################################################################################
timestamp = InvCall()  ## Grab time stamp
print(timestamp)
fmt = "%Y-%m-%d %H:%M:%S"
# local time
#t = datetime.datetime.fromtimestamp(int(timeInfo[1])/1000)
#print (t.strftime(fmt)) # prints 2012-08-28 02:45:17
# utc time
t_utc = datetime.datetime.utcfromtimestamp(int(timestamp)/1000)
t_utc_formatted = t_utc.strftime(fmt)
APIcall(timestamp,t_utc_formatted)